In [34]:
### Thank you to Dr. Arrington for providing a lot of framework for this code ###

# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import pymongo
from webdriver_manager.chrome import ChromeDriverManager

In [35]:
# Set up Splinter
executable_path = {'executable_path': 'C:/Users/Joshua/Downloads/chromedriver_win32/chromedriver.exe'}
browser = Browser('chrome', **executable_path)

## Visit the NASA Mars news site 

In [36]:
# Visit the Mars news sit
mars_news_url = f'https://redplanetscience.com'
browser.visit(mars_news_url)

# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time = 1)

True

In [37]:
# Conver the browser html to a soup object
html = browser.html
news_soup = soup(html, 'html.parser')

slide_elem = news_soup.select_one('div.list_text')

In [38]:
# Display the current title content
slide_elem.find('div', class_='content_title')

<div class="content_title">7 Things to Know About the Mars 2020 Perseverance Rover Mission</div>

In [39]:
# Use the parent element to find the first tag and save it as 'news_title'
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

'7 Things to Know About the Mars 2020 Perseverance Rover Mission'

In [40]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

"NASA's next rover to the Red Planet is slated to launch no earlier than July 30. These highlights will get you up to speed on the ambitious mission."

## JPL Space Images Featured Image 

In [41]:
# Visit featured image url
featured_image_url = 'https://spaceimages-mars.com'
browser.visit(featured_image_url)

In [43]:
# Find and click the full image button
full_image_link = browser.find_by_tag('button')[1]
full_image_link.click()

In [45]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')
#print(img_soup.prettify())

In [47]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars2.jpg'

In [48]:
# Use the base url to create an absolute url
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

## Mars Facts

In [56]:
# Use 'pd.read_html' to pull the data from the Mars-Earth comparison section
# hint: use index 0 to find the table
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.head()

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"


In [57]:
df.columns=['Description', 'Mars', 'Earth']
df.set_index('Description', inplace=True)
df

,Mars,Earth
Description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [58]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

## Hemispheres 

In [59]:
mars_hem_url = f'https://marshemispheres.com'
browser.visit(mars_hem_url)

In [60]:
# Create a list to hold the images and titles
hemisphere_image_urls = []

# Get a list of all of the hemispheres
links = browser.find_by_css('a.product-item img')

# Next, loop through those links, click the links, find the sample anchor, return the href
for i in range(len(links)):
    # hemisphere info dictionary
    hemisphereInfo = {}
    
    # We have to find the elements on each loop to avoid a stale element exception
    browser.find_by_css('a.product-item img')[i].click()
    
    # Next, we find the Sample image anchor tag and extract the href
    sample = browser.links.find_by_text('Sample').first
    hemisphereInfo['img_url'] = sample['href']
    
    # Get Hemisphere title
    hemisphereInfo['title'] = browser.find_by_css('h2.title').text
    
    # Append hemisphere object to list
    hemisphere_image_urls.append(hemisphereInfo)
    
    # Finally, we navigate backwards
    browser.back()

In [61]:
hemisphere_image_urls

[{'img_url': 'https://marshemispheres.com/images/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [62]:
browser.quit()